In [1]:
import numpy as np
from numpy.random import default_rng
import optimizers
import operators
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
import utils
import pywt

# Quantifying Uncertainty in a Reconstruction

Assume we have a great, accurate inversion algorithm and with that, we have computed some solution $x^{\ast}$ from the data observation $y$ in the inverse problem 
$$y = A(x) + \varepsilon$$
> How certain are we that $x^{\ast}$ is a representative solution? 
> How large are the errors that we could have made because of the stochastic nature of the noise $\varepsilon$? 

For instance, consider the following example of making two independent CT scans of the same object:

In [ ]:
import matplotlib.gridspec as gridspec

rng = default_rng(2452)

fig = plt.figure(tight_layout=True,figsize=(16,8))
gs = gridspec.GridSpec(2, 5)

dim=100
phantom = utils.get_phantom(dim)
theta = np.linspace(0,180, endpoint = False, num=128)
R = operators.Radon(theta=theta)
sinogram = R(phantom)

# two different instantiations of the randomly distributed noise
sigma = 0.01
noise1 = sigma*rng.normal(size=sinogram.shape)
noise2 = sigma*rng.normal(size=sinogram.shape)
y1 = sinogram + noise1
y2 = sinogram + noise2

# compute the minimizer
mu_tv = 0.005
opt1 = optimizers.split_Bregman_TV(R,y1,np.zeros(phantom.shape),1,mu_tv,max_it=25,verbosity=0,energy_fun=lambda x:np.sqrt(np.sum((R(x)-y1)**2))/dim**2)
opt2 = optimizers.split_Bregman_TV(R,y2,np.zeros(phantom.shape),1,mu_tv,max_it=25,verbosity=0,energy_fun=lambda x:np.sqrt(np.sum((R(x)-y2)**2))/dim**2)
rec1 = opt1.solve()
rec2 = opt2.solve()

# plot results
ax = [fig.add_subplot(gs[:, 0:2])]
for i in range(6): ax.append(fig.add_subplot(gs[i%2,2+i//2]))
ax[0].imshow(phantom, cmap='gray',vmin=0,vmax=1), ax[0].set_title('Ground truth')
ax[1].imshow(y1, cmap='gray'), ax[1].set_title('Noisy scan 1')
ax[2].imshow(y2, cmap='gray'), ax[2].set_title('Noisy scan 2')
ax[3].imshow(rec1, cmap='gray'), ax[3].set_title('Reconstruction from scan 1')
ax[4].imshow(rec2, cmap='gray'), ax[4].set_title('Reconstruction from scan 2')
ax[5].imshow(rec1[dim//2-10:dim//2+10,dim//2-10:dim//2+10], cmap='gray', vmin=0, vmax=0.5), ax[5].set_title('Detail rec. 1 (Contrast enhanced)')
ax[6].imshow(rec2[dim//2-10:dim//2+10,dim//2-10:dim//2+10], cmap='gray', vmin=0, vmax=0.5), ax[6].set_title('Detail rec. 2 (Contrast enhanced)')

for axes in ax: axes.axis('off')

In order to answer, we have to give some probabilistic meaning to the distribution of possible solutions $x$, noise $\varepsilon$ and the observed data $y$. We assume that both $x$ and $\varepsilon$ are instantiations of random variables with certain probability distributions. For the marginal distribution of possible solutions, we write $x \sim \pi_0$. The probability of observing $y = A(x)+\varepsilon$ given some $x$ is denoted $\pi_{y|x}$.
For our purposes we assume that those random variables have an absolutely continuous univariate distribution. The can be "discribed" by a probability desnity function in the following way:

**1-d probability density function:**
$$
P[a\leq X \leq b]=\int_a^b f_X(x)dx \quad (1)
$$
$$
\mathbb{E}[X]=\int_{\mathbb{R}} x f_X(x)dx \quad (2)
$$
$$
Var[X]=\int_{\mathbb{R}} (\mathbb{E}[X]-x)^2 f_X(x)dx \quad (3)
$$

### &#128221; <span style="color:darkorange"> Question </span>
What properties should a probability density function have?\
How dose a multidimensional probability density function look like compared to (1-3) ?


$\pi_0$ is called the **prior distribution**, it encodes prior information, assumptions or belief about the possible nature of our reconstruction $x$. Assume that $\pi_0$ has a probability density function (pdf) $p_0(x)$.

The density $p(\cdot|x)$ of the distribution $\pi_{y|x}$ is called the **likelihood**.

Once we make an observation $y$, by **Bayes' law** the **posterior distribution** of $x$ after seeing $y$ has the density
$$ p(x|y) = \frac{p(y|x)p_0(x)}{p(y)} \propto p(y|x) p_0(x). $$

### Prior distribution
Examples
- If we believe our solution $x$ should be close to some a priori known value $\hat x$, then our prior might look something like $p_0(x) = \exp(-\frac{1}{2\sigma^2}||x-\hat x||_2^2)/Z$. The normalization constant $Z$ ensures that $p_0$ is a valid pdf, we omit it from now on and write "$\propto$" for proportionality. According to this prior density, values closer to $\hat x$ are assigned larger probability values, since they are assumed to be more likely.
- If we believe that $x$ is likely to have little edges/contrast jumps, we might pick a prior like $p_0(x) \propto \exp(-\mathrm {TV}(x)) = \exp(- \lVert \nabla x \rVert_1)$.
- In general, distributions of the form $ p_0(x) \propto \exp(-G(x))$ for any $G$ are called **Gibbs priors**.

### Noise distribution & likelihood function
- Homoschedastic Gaussian observation noise $\varepsilon \sim \mathrm{N}(0,I)$ corresponds to a likelihood of the form $ p(y|x) \propto \exp(- ||y-A(x)||_2^2 ) $.
- If the observed variable $y$ follows a Poisson distribution $y \sim \mathrm{Pois}(A(x))$, then $p(y|x) \propto \frac 1{y!} \exp(-A(x)) A(x)^y $ for all $y \in \mathbb{N}$.

### &#128221; <span style="color:darkorange"> Question </span>
What is the relation of the Maximum likelihood Estimatior
$$ \arg\max_x p(x|y)  $$
for $p_0(x)\propto\exp(- \lambda\mathrm {TV}(x))$ and $p(y|x)\propto\exp(- ||y-A(x)||_2^2 )$ and $\lambda>0$ to classical variational inversion?

## 1D Example
As a simple example, consider $x\in \mathbb{R}$ just to be a number that we want to estimate from the inverse problem $y = a \cdot x + \varepsilon$ with $a = 4$. We make one observation of the random variable $y$.

We assume that the noise is distributed like $\varepsilon \sim \mathrm N(0,\sigma^2)$ and a prior $x \sim \mathrm N(\mu_{0}, 1)$.\
Play with the sliders below to get some intuition on how the posterior changes depending on the observation and the parameters in prior and likelihood.

In [2]:
def plot_1dposterior(sigma, mu0, y):
    fig,ax = plt.subplots(1, 3, figsize=(15,5))
    # prior
    t = np.linspace(-5,5,500)
    ax[0].plot(t,1/np.sqrt(2*np.pi)*np.exp(-(t-mu0)**2/2))
    ax[0].set_title('Prior distribution p(x)')

    # likelihood
    a = 4               # the "forward model"
    ax[1].plot(t,1/(np.sqrt(2*np.pi)*sigma)*np.exp(-(y-a*t)**2/(2*sigma**2)))
    ax[1].set_title('Likelihood function p(y|x)')

    # posterior distribution
    ax[2].plot(t,np.exp(-(t-mu0)**2/2 -(y-a*t)**2/(2*sigma**2)))
    ax[2].scatter(1/(1+a**2/sigma**2)*(mu0 + a/sigma**2 * y),0,c='r',marker='x')
    ax[2].set_yticks([])
    ax[2].set_title('Posterior distribution p(x|y)')

interactive_plot = interactive(plot_1dposterior, sigma = (0.1,4.0,0.1), mu0 = (-2.0,2.0,0.1), y = (-10.0,10.0,0.1))
display(interactive_plot)

interactive(children=(FloatSlider(value=2.0, description='sigma', max=4.0, min=0.1), FloatSlider(value=0.0, de…

## Evaluating the posterior
Let's consider a simple **image denoising** example where we measure a noisy image from
$$ y = x + \varepsilon $$
where $x$ is some image. Suppose the errors $\varepsilon$ are again distributed as mean-zero Gaussian with standard deviation $\sigma$.

In order to regularize, we assume $x$ is sparse in some basis $D$ (you've seen the example with $D$ a Wavelet transform in Tutorial 2).\
The corresponding Gibbs prior has density
$$ p_0(x) \propto \exp(-|| Dx ||_1). $$

We want to compute some statistics of the posterior distribution:
- The posterior mean $\mu_{x|y} := \mathbb E_{x|y}[x]$
- The posterior (pixel-wise) variance $ \sigma^2_{x|y} := \mathbb E_{x|y}[(x- \mu_{x|y})^2] $

In order to compute $\mu_{x|y}$ and $\sigma^2_{x|y}$, we need to draw samples from the posterior $\pi_{x|y}$. We use a Markov chain Monte Carlo algorithm based on Langevin diffusion, which iterates
$$ \begin{array}{l} \xi^{k+1} \sim \mathrm N (0, I), \\
X_{k+1} = X_k - \tau \nabla V(X_k) + \sqrt{2\tau} \xi^{k+1}.
\end{array} $$
Here $ V $ is the negative log-density of the posterior: $V(x) = -\log p_{x|y}(x)$.

### &#128221; <span style="color:darkorange"> Tasks </span>
The following code computes the minimizer $\hat x = \min V(x)$, initializes $X_0 = \hat x$ and then runs the above Langevin sampling algorithm.
- Compute an estimator for the posterior mean and plot it in ax[0,2].
- Extract two arbitrary samples from Markov chain and plot them in ax[1,0] and ax[1,1]. How do the samples differ qualitatively from $\hat x$ and the posterior mean?
- Can you come up with an efficient way to compute an estimator of $\sigma^2_{x|y}$? Plot $\sigma^2_{x|y}$ for each pixel in ax[1,2].

In [3]:
sigma = 0.1
lamda = 30
t = sigma**2

dim=256
phantom = utils.get_phantom(dim)

wl = pywt.Wavelet('haar')
y = phantom + sigma*rng.normal(size=(dim,dim))
opti = optimizers.ista_wavelets(operators.Identity(), np.zeros_like(phantom), y, wave = wl, t=1, lamda=lamda*sigma**2,  verbosity=0, max_it=1)
opti.solve()
x = np.copy(opti.x)

fig,ax=plt.subplots(2,3,figsize=(15,10))
# Look here to see how to plot !!!
ax[0,0].imshow(y, cmap='gray', vmin=0, vmax = 1)
ax[0,0].set_title('Noisy Observation')
ax[0,1].imshow(x, cmap='gray', vmin=0, vmax = 1)
ax[0,1].set_title('Denoised minimizer $\hat x$')

n_samples = 1000
log_post_dens = np.zeros((n_samples,))

running_sum = np.zeros_like(x)
running_square_sum = np.zeros_like(x)
x_hist = []

for i in range(n_samples):
    # Langevin sampling step
    z = x - t /sigma**2 * (x - y) + np.sqrt(2*t)*rng.normal(size=(dim,dim))
    wv_coeffs, slices = pywt.coeffs_to_array(pywt.wavedec2(z, wavelet=wl, mode='periodization'))
    wv_coeffs_shr = operators.soft_shrinkage(wv_coeffs, lamda * t)
    # For each iteration x now contains a sample of the posterior distribution generated by the Langevin sampling algorithem
    x = pywt.waverec2(pywt.array_to_coeffs(wv_coeffs_shr, slices, output_format='wavedec2'), wavelet=wl, mode='periodization')


NameError: name 'rng' is not defined